In [1]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
import os, torch
import numpy as np
from sklearn.metrics import *
import mplhep as hep

/Users/lauraboggia/VSCode_projects/TranAD/.conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.data_loader import convert_to_windows_test, load_dataset
from main import backprop
from src.utils import load_model

In [3]:
# parameters
modelname = 'iTransformer'
dataset = 'IEEECIS_new'
checkpoints_path = 'iTransformer_IEEECIS_new/n_window100_steps50/checkpoints/model.ckpt'
n_window = 100
step_size = 50
feats = 62

In [4]:
train_loader, test_loader, labels, ts_lengths = load_dataset(dataset)
model, optimizer, scheduler, epoch, accuracy_list = load_model(modelname, train_loader.dataset.shape[1], checkpoints_path)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'trainable params: {trainable_params}')

training set shape: (16708, 62)
test set shape: (190, 62)
labels shape: (190, 62)
Creating new model: iTransformer
trainable params: 255590


In [5]:
## Prepare data
trainD, testD = next(iter(train_loader)), next(iter(test_loader))
trainD, testD = trainD.to(torch.float64), testD.to(torch.float64)  # necessary because model in double precision, data should be as well
trainO, testO = trainD, testD
if model.name in ['Attention', 'DAGMM', 'USAD', 'MSCRED', 'CAE_M', 'GDN', 'MTAD_GAT', 'MAD_GAN', 'iTransformer'] or 'TranAD' in model.name: 
    train_test, _ = convert_to_windows_test(trainO, model, labels=None, w_size=n_window)  # use non-overlapping windows for testing, need this for POT
    testD, labels  = convert_to_windows_test(testD, model, labels, w_size=n_window)  # later use this: args.n_window)  # use non-overlapping windows for testing

print(train_test.shape, testD.shape, labels.shape)

torch.Size([168, 100, 62]) torch.Size([2, 100, 62]) (200, 62)


In [6]:
torch.zero_grad = True
model.eval()
loss, y_pred = backprop(0, model, testD, feats, optimizer, scheduler, training=False)
lossT, _ = backprop(0, model, train_test, feats, optimizer, scheduler, training=False)  # need anomaly scores on training data for POT

RuntimeError: mat1 and mat2 shapes cannot be multiplied (124x100 and 10x124)

In [7]:
plt.style.use([hep.style.ROOT, hep.style.firamath])
plt.rcParams['lines.markersize'] = 5
plt.rcParams['lines.linewidth'] = 3

In [8]:
def smooth(y, box_pts=1):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [ ]:
def plot_labels(path, name, y_pred, y_true):
	os.makedirs(path, exist_ok=True)
	fig, ax1 = plt.subplots()
	ax1.plot(smooth(y_pred), 'g.', label='Predicted anomaly')
	ax1.plot(y_true, '--', alpha=0.2)
	ax1.fill_between(np.arange(y_true.shape[0]), y_true, color='blue', alpha=0.3, label='True anomaly')
	plt.legend()
	ax1.set_xlabel('Timestamp')
	ax1.set_ylabel('Labels')
	plt.show()
	# plt.close()
